In [1]:
import os
import openai
import sqlite3
openai.api_key = os.environ["OPENAI_APIKEY"]

In [2]:
def prompt(prompt):
    response = openai.Completion.create(
        model="text-davinci-003", # "code-davinci-002",
        prompt=prompt,
        temperature=0,
        max_tokens=500,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response.choices[0].text

In [3]:
def in_context_of(uri):
    def inner(*, ask_to):
        return prompt(f"""
            {read_schema(uri)}

            {ask_to}
        """)
    return inner

In [4]:
# code courtesy of ChatGPT-3

def create_example_sqlite_db(path):
    # create a connection to the database
    conn = sqlite3.connect(path)

    # create a cursor object to execute SQL commands
    cursor = conn.cursor()

    # create the Players table
    cursor.execute('''CREATE TABLE Players (
                        id INTEGER PRIMARY KEY,
                        name TEXT
                    )''')

    # create the Seasons table with a foreign key constraint on the player_id column
    cursor.execute('''CREATE TABLE Seasons (
                        id INTEGER PRIMARY KEY,
                        year INTEGER,
                        player_id INTEGER,
                        total_score INTEGER,
                        total_fauls INTEGER,
                        FOREIGN KEY(player_id) REFERENCES Players(id)
                    )''')

    # create the Games table with a foreign key constraint on the player_id column
    cursor.execute('''CREATE TABLE Games (
                        id INTEGER PRIMARY KEY,
                        player_id INTEGER,
                        fauls INTEGER,
                        score INTEGER,
                        FOREIGN KEY(player_id) REFERENCES Players(id)
                    )''')

    # commit the changes to the database
    conn.commit()

    # close the cursor and the connection
    cursor.close()
    conn.close()

    
def read_schema(uri):
    # Connect to the database
    conn = sqlite3.connect(uri)
    cursor = conn.cursor()

    # Get a list of all the tables in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Loop over each table and get its schema
    schema_str = ""
    for table in tables:
        cursor.execute(f"PRAGMA table_info({table[0]})")
        table_info = cursor.fetchall()

        # Add the table name to the schema string
        schema_str += f"\n\nTable: {table[0]}\n"

        # Loop over each column in the table and add it to the schema string
        for column in table_info:
            col_name = column[1]
            col_type = column[2]
            is_pk = " (PK)" if column[5] == 1 else ""
            schema_str += f"{col_name} {col_type}{is_pk}\n"

        # Get a list of foreign keys for the table
        cursor.execute(f"PRAGMA foreign_key_list({table[0]})")
        foreign_keys = cursor.fetchall()

        # Add foreign key information to the schema string
        if foreign_keys:
            schema_str += "\nForeign keys:\n"
            for key in foreign_keys:
                schema_str += f"{key[3]} REFERENCES {key[2]}({key[4]})\n"

    # Close the database connection
    conn.close()

    # Return the schema string
    return schema_str

In [5]:
create_example_sqlite_db('example.db')

In [6]:
%%time
describe_db = ("Analyze this schema and tell some interesting insights about "
               "it as well as generate some interesting SQL queries.")
print(in_context_of('example.db')(ask_to=describe_db))


This schema is used to store information about players, their seasons, and their individual games. It allows us to track a player's performance over time and compare it to other players.

Some interesting insights that can be gleaned from this schema include:

- The total score and total fouls for each season for each player.
- The average score and average fouls for each season for each player.
- The total score and total fouls for each game for each player.
- The average score and average fouls for each game for each player.

Some interesting SQL queries that can be generated from this schema include:

- Find the total score and total fouls for each season for each player:
SELECT p.name, s.year, SUM(g.score) AS total_score, SUM(g.fouls) AS total_fouls
FROM Players p
INNER JOIN Seasons s ON p.id = s.player_id
INNER JOIN Games g ON p.id = g.player_id
GROUP BY p.name, s.year

- Find the average score and average fouls for each season for each player:
SELECT p.name, s.year, AVG(g.score)

In [7]:
%%time
translate_to = lambda vendor: (f"Translate that schema to code that can be used "
                                     f"to build schema in {vendor}.")
print(in_context_of('example.db')(ask_to=translate_to("mysql")))


CREATE TABLE Players (
    id INTEGER PRIMARY KEY,
    name TEXT
);

CREATE TABLE Seasons (
    id INTEGER PRIMARY KEY,
    year INTEGER,
    player_id INTEGER,
    total_score INTEGER,
    total_fauls INTEGER,
    FOREIGN KEY (player_id) REFERENCES Players(id)
);

CREATE TABLE Games (
    id INTEGER PRIMARY KEY,
    player_id INTEGER,
    fauls INTEGER,
    score INTEGER,
    FOREIGN KEY (player_id) REFERENCES Players(id)
);
CPU times: user 8.24 ms, sys: 3.91 ms, total: 12.2 ms
Wall time: 7.61 s


In [8]:
%%time
generate_marmaidjs_diagram = ("Generate marmaid.js code that shows each table as node and "
                              "foreign key relationships as edges between the nodes.")
in_context_of('example.db')(ask_to=generate_marmaidjs_diagram)

CPU times: user 5.78 ms, sys: 2.88 ms, total: 8.67 ms
Wall time: 2.15 s


'\ngraph LR\nPlayers[Players] --> Seasons[Seasons]\nSeasons[Seasons] --> Games[Games]'